## 1. Parse parameter 

In the following code, the FLAGS is not tf.app.flags.FLAGS, it just a globel variable, which will not pass to main() as paramter, but it will be directly used by functions

In [3]:
import argparse

FLAGS = None

parser = argparse.ArgumentParser()

parser.add_argument(
      '--learning_rate',
      type=float,
      default=0.01,
      help='Initial learning rate.'
)

parser.add_argument(
      '--num_epochs',
      type=int,
      default=2,
      help='Number of epochs to run trainer.'
)

FLAGS, unparsed = parser.parse_known_args(['--learning_rate', '0.03', '--foo', '--badger', 'BAR', 'spam'])

print(FLAGS)
print(FLAGS.learning_rate)
print(unparsed)

tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Namespace(learning_rate=0.03, num_epochs=2)
0.03
['--foo', '--badger', 'BAR', 'spam']


## 2. FLAGS

The tf.app.flags.FLAGS is used to pass parameters through the argument of main(), the value of FLAGS.max_steps and FLAGS.num_gpus become 50 and 20 (initialized as 100 and 1).

In [1]:
import tensorflow as tf

FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_integer('max_steps', 100,
                            """Number of batches to run.""")
tf.app.flags.DEFINE_integer('num_gpus', 1,
                            """How many GPUs to use.""")


def main(argv=None):
    print(FLAGS.max_steps)
    print(FLAGS.num_gpus)

if __name__ == '__main__':
  # the first param for argv is the program name
  tf.app.run(main=main, argv=['tensorflow_read_data', '--max_steps', '50', '--num_gpus', '20'])

50
20


SystemExit: 

C:\Users\Boyuan.Feng\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
